In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',None)
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer


In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test0=pd.read_csv('test.csv')

# Data Fields

**datetime** - hourly date + timestamp  

**season**-  1 = spring, 2 = summer, 3 = fall, 4 = winter

**holiday** - whether the day is considered a holiday

**workingday** - whether the day is neither a weekend nor holiday

**weather** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

**temp** - temperature in Celsius

**atemp**- "feels like" temperature in Celsius

**humidity** - relative humidity

**windspeed** - wind speed

**casual** - number of non-registered user rentals initiated

**registered** - number of registered user rentals initiated

**count** - number of total rentals


In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
df=train.copy()

Remove irrelevant features ; casual and registered

In [ ]:
df.drop(columns=['casual','registered'],inplace=True)


In [ ]:
df.info()

Convert datetime to hour,weekday,month,year

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.strftime('%a')
df['month'] = df['datetime'].dt.strftime('%b')
df['year'] = df['datetime'].dt.year
df.drop(columns=['datetime'],inplace=True)
test['datetime'] = pd.to_datetime(test['datetime'])
test['hour'] = test['datetime'].dt.hour
test['day'] = test['datetime'].dt.strftime('%a')
test['month'] = test['datetime'].dt.strftime('%b')
test['year'] = test['datetime'].dt.year
test.drop(columns=['datetime'],inplace=True)

In [ ]:
df['season'] = df['season'].map({1: 'Spring', 2: 'Summer', 3: 'Fall,', 4: 'Winter'})
df['weather'] = df['weather'].map({1: 'Clear', 2: 'Cloudy', 3: 'Light Rain', 4: 'Heavy Rain'})
df['holiday'] = df['holiday'].map({0: 'No', 1: 'Yes'})
df['workingday'] = df['workingday'].map({0: 'No', 1: 'Yes'})
test['season'] = test['season'].map({1: 'Spring', 2: 'Summer', 3: 'Fall,', 4: 'Winter'})
test['weather'] = test['weather'].map({1: 'Clear', 2: 'Cloudy', 3: 'Light Rain', 4: 'Heavy Rain'})
test['holiday'] = test['holiday'].map({0: 'No', 1: 'Yes'})
test['workingday'] = test['workingday'].map({0: 'No', 1: 'Yes'})

In [ ]:
df.head()

In [ ]:
categorical=['season','holiday','workingday','weather','hour','day','month','year']
numeric=['temp','atemp','humidity','windspeed']

 Plot categorical

In [ ]:
plt.figure(figsize=(20,20))
for index,feature in enumerate(categorical):
  plt.subplot(3, 4, index+1)
  sns.barplot(x=feature, y='count', data=df)
  plt.xlabel(f'{feature}')
  plt.ylabel('Categorical Variable')
  plt.title(f'{feature} vs count')
plt.show()

Plot Numeric

In [ ]:
plt.figure(figsize=(20,20))
for index,feature in enumerate(numeric):
  plt.subplot(3, 4, index+1)
  sns.scatterplot(x=feature, y='count', data=df)
  plt.xlabel(f'{feature}')
  plt.ylabel('Numeric Variable')
  plt.title(f'{feature} vs count')
plt.show()

In [ ]:
sns.displot(df['count'], kde=True, color='blue')
plt.title('Distribution of Count')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Right side skewed

Tranformation of count distribution

In [ ]:
df['log_count'] = np.log1p(df['count'])
sns.histplot(df['log_count'], kde=True, color='blue')
plt.title('Log-transformed Distribution of Target Variable')
plt.xlabel('Log-transformed Target Variable Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
df.drop(columns=['count'],inplace=True)

Outlier checking for numeric

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[numeric])
plt.title('Boxplot of Numeric Variables')
plt.xlabel('Variable')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.show()

windspeed has outliers

remove all outliers

In [ ]:
drop_idx=df[(df['atemp']>20) & (df['atemp'] <48) & (df['temp']>10) & (df['temp']<20)].index
clean_df=df.drop(drop_idx)

In [ ]:
clean_df.head()

In [ ]:
sns.boxplot(data=clean_df['count'])
plt.title('Boxplot of Count')
plt.xlabel('Count')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.show()

checking correlation

In [ ]:
sns.heatmap(clean_df[numeric].corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

temp and atemp is correlated,so we can remove it

In [ ]:
clean_df.drop(columns=['atemp'],inplace=True)
test.drop(columns=['atemp'],inplace=True)

convert categorical into numeric

In [ ]:
final_df = pd.get_dummies(clean_df, columns=categorical, drop_first=True)
test = pd.get_dummies(test, columns=categorical, drop_first=True)

In [ ]:
final_df = final_df.astype(int)
test = test.astype(int)

splitting and training

In [ ]:
print('final_df shape:', final_df.shape)
print('train shape:', train.shape)
print('test shape:',  test.shape)

In [ ]:
final_df.head()

In [ ]:
final_df['log_count'].value_counts()

In [ ]:
X=final_df.drop(columns=['log_count'])
y=final_df['log_count']



Scaling using Standardscaling

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_train_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, train_size=0.8, test_size=0.2,random_state=0)

In [ ]:
def rmsle(y,pred):
  log_y=np.log1p(y)
  log_pred=np.log1p(pred)
  squared_error=(log_y-log_pred)**2
  rmsle=np.sqrt(np.mean(squared_error))
  return rmsle

def evaluate(reg_cls,params=None):
  reg=reg_cls()
  if params:
    reg=GridSearchCV(reg, param_grid=params, refit=True)
  reg.fit(X_train,y_train)
  pred=reg.predict(X_test)
  y_test_exp=np.expm1(y_test)
  pred_exp=np.expm1(pred)
  print('\n', reg_cls)
  if params:
    print(reg.best_params_)
    reg=reg.best_estimator_
  print(rmsle(y_test_exp,pred_exp))
  return reg, pred_exp

In [ ]:
Ir_reg, pred_Ir=evaluate(LinearRegression)
rg_reg, pred_rg=evaluate(Ridge)
ls_reg, pred_ls=evaluate(Lasso)
rf_reg, pred_rf=evaluate(RandomForestRegressor)
gb_reg,pred_gb=evaluate(GradientBoostingRegressor)
xg_reg, pred_xg=evaluate(XGBRegressor)
lg_reg, pred_1g=evaluate(LGBMRegressor)

params={'n_estimators': [100*i for i in range(1,6)]}
xg_reg, pred_xg=evaluate(XGBRegressor, params)
lg_reg, pred_1g=evaluate(LGBMRegressor, params)

best model is LGBM

In [ ]:
predictions = lg_reg.predict(test)
original_predictions = np.expm1(predictions)
predictions_df = pd.DataFrame({'datetime': test0['datetime'], 'count': original_predictions})
predictions_df.to_csv('predictions.csv', index=False)